In [2]:
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
import plotly.express as px
import pandas as pd
init_notebook_mode(connected=True)

trace0 = Scatter(
  x=[1, 2, 3, 4],
  y=[10, 15, 13, 17]
)
trace1 = Scatter(
  x=[1, 2, 3, 4],
  y=[16, 5, 11, 9]
)

iplot([trace0, trace1])

### Data Load & Preprocess

In [3]:
df = pd.read_csv("C:/Users/avallejof/Downloads/all_bikez/all_bikez_raw.csv", low_memory=False)
df.head()

,Model,Year,Category,Rating,Displacement,Engine type,Engine details,Power,Top speed,Bore x stroke,...,Emission details,Rear percentage of weight,Oil filter,Battery,Belt teeth,Belt width,Pulley teeth,Chain size,Factory warranty,Service interval
0,AJP GALP 50 Enduro,2005,Enduro / offroad,3.7 Check out the detailed rating of off-roa...,49.9 ccm (3.04 cubic inches),"Single cylinder, two-stroke",NaN,9.5 HP (6.9 kW)),NaN,40.0 x 39.7 mm (1.6 x 1.6 inches),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AJP GALP 50 R,2005,Sport,3.6 Check out the detailed rating of racing ...,49.9 ccm (3.04 cubic inches),"Single cylinder, two-stroke",NaN,9.5 HP (6.9 kW)),NaN,40.0 x 39.7 mm (1.6 x 1.6 inches),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AJP GALP 50 Supermotard,2005,Super motard,1.9 Check out the detailed rating of off-roa...,49.9 ccm (3.04 cubic inches),"Single cylinder, two-stroke",NaN,9.5 HP (6.9 kW)),NaN,40.0 x 39.7 mm (1.6 x 1.6 inches),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AJP PR3 125 Enduro,2015,Enduro / offroad,2.1 Check out the detailed rating of off-roa...,124.0 ccm (7.57 cubic inches),"Single cylinder, four-stroke",NaN,12.5 HP (9.1 kW)) @ 12600 RPM,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AJP PR3 125 Enduro Pro,2015,Enduro / offroad,Do you know this bike?Click here to rate it. W...,124.0 ccm (7.57 cubic inches),"Single cylinder, four-stroke",NaN,12.5 HP (9.1 kW)) @ 12600 RPM,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Model & brand separation

In [4]:
brand_df = pd.read_csv("C:/Users/avallejof/Downloads/all_bikez/bikez_brands.csv", low_memory=False)
brand_list = '|'.join(list(brand_df.Brand)).replace("-","\-")
brand_df.head()

,Brand
0,Acabion
1,Access
2,Ace
3,Adiva
4,Adler


We keep only the brands with more than 200 models as they are probably more relevant

In [5]:
# df.apply(lambda x: x['Model'].str.extractall('|'.join(brand_list)), axis=1)
df["Brand"] = df.Model.str.extract(f"({brand_list})")[0]
(df.Brand.value_counts()[df.Brand.value_counts() >200]).index

Index(['Yamaha ', 'Honda ', 'Suzuki ', 'Kawasaki ', 'Harley-Davidson ', 'BMW ',
       'KTM ', 'Triumph ', 'Ducati ', 'Aprilia ', 'Moto Guzzi ', 'Husqvarna ',
       'Kymco ', 'MV Agusta ', 'Benelli ', 'Keeway ', 'GAS GAS ', 'Derbi ',
       'Sherco ', 'Sym ', 'Peugeot ', 'Vespa ', 'Hyosung ', 'Indian ',
       'Enfield ', 'Beta ', 'TM Racing ', 'NSU ', 'Rieju ', 'Piaggio ',
       'Gilera '],
      dtype='object')

In [6]:
df = df[df.Brand.isin(df.Brand.value_counts().head(10).index)]
df.Brand = df.Brand.str.strip()
df.Brand.value_counts()

Yamaha             3414
Honda              3318
Suzuki             2440
Kawasaki           2320
Harley-Davidson    1636
BMW                1145
KTM                1057
Triumph             874
Ducati              832
Aprilia             804
Name: Brand, dtype: int64

#### Selecting and Cleaning Variables

In [7]:
df = df.loc[:,["Brand","Model","Year","Displacement",
            "Power","Top speed", "Fuel consumption", "Dry weight"]]


In [8]:
countries = {
"Yamaha": "Japan",
"Honda": "Japan",
"Suzuki": "Japan",
"Kawasaki": "Japan",
"Harley-Davidson": "USA",
"BMW": "Germany",
"KTM": "Austria",
"Triumph": "United Kingdom",
"Ducati": "Italy",
"Aprilia": "Italy",
}
df["Country"] = df.apply(lambda x:countries[x.Brand], axis=1)
df["Country"]

666      Italy
667      Italy
668      Italy
669      Italy
670      Italy
         ...  
38012    Japan
38013    Japan
38014    Japan
38015    Japan
38016    Japan
Name: Country, Length: 17840, dtype: object

In [9]:
df["Power"] = df["Power"].apply(lambda x: float(x.split("HP")[0]) if type(x) == str else float(x))
df["Displacement"] = df["Displacement"].apply(lambda x: float(x.split("ccm")[0]) if type(x) == str else float(x))
df["Fuel consumption"] = df["Fuel consumption"].apply(lambda x: float(x.split("litres")[0]) if type(x) == str else float(x))
df["Top speed"] = df["Top speed"].apply(lambda x: float(x.split("km/h")[0]) if type(x) == str else float(x))
df["Dry weight"] = df["Dry weight"].apply(lambda x: float(x.split("kg")[0]) if type(x) == str else float(x))
df = df.loc[df["Fuel consumption"] < 16]

df = df.rename(columns={
    "Power":"Power (CV)",
    "Displacement":"Displacement (cc)",
    "Fuel consumption":"Fuel consumption (l/100km)",
    "Top speed":"Top speed (km/h)",
    "Dry weight":"Dry weight (kg)"})

df.head()

,Brand,Model,Year,Displacement (cc),Power (CV),Top speed (km/h),Fuel consumption (l/100km),Dry weight (kg),Country
757,Aprilia,Aprilia Dorsoduro 900,2021,896.1,95.2,NaN,5.60,195.0,Italy
758,Aprilia,Aprilia Dorsoduro 900,2022,896.1,95.2,NaN,5.60,195.0,Italy
767,Aprilia,Aprilia ETX 125,1999,125.0,12.2,98.0,6.10,NaN,Italy
829,Aprilia,Aprilia Moto 6.5,1999,649.0,42.0,160.0,6.10,NaN,Italy
841,Aprilia,Aprilia Pegaso 650,1993,652.0,48.0,160.0,5.22,160.0,Italy


#### Saving the df

In [11]:
df.reset_index(drop=True).to_pickle("../data/moto_df.pkl")

In [12]:
#### Plotting

In [181]:
fig = px.scatter(
    df, x="Power", y="Fuel consumption", color="Brand",
    width=1200, height=700,
    trendline= "lowess", trendline_scope="overall", trendline_color_override="black",
    color_discrete_sequence=px.colors.qualitative.G10, hover_name="Model")
fig.show()

In [183]:
fig = px.scatter(
    df, x="Displacement", y="Power", color="Brand",
    width=1200, height=700,
    trendline= "lowess",
     trendline_scope="overall", trendline_color_override="black",
    color_discrete_sequence=px.colors.qualitative.G10, hover_name="Model")
fig.show()

In [184]:
fig = px.box(
    df, x="Brand", y="Power", color="Brand",
    # df, x="Brand", y=df["Fuel consumption"], color="Brand",
    width=1200, height=700, points="all",
    hover_data=["Fuel consumption", "Power"],
    color_discrete_sequence=px.colors.qualitative.G10, hover_name="Model")
# fig.update_traces(hovertemplate = "Brand:%{x} <br>Consumption/power: %{y} </br>(Fuel consumption, Power) : %{customdata}")
fig.show()

In [188]:
df.drop("Consumption/power",axis=1)

,Brand,Model,Year,Displacement,Power,Top speed,Fuel consumption,Dry weight
757,Aprilia,Aprilia Dorsoduro 900,2021,896.1,95.2,NaN,5.60,195.0
758,Aprilia,Aprilia Dorsoduro 900,2022,896.1,95.2,NaN,5.60,195.0
767,Aprilia,Aprilia ETX 125,1999,125.0,12.2,98.0,6.10,NaN
829,Aprilia,Aprilia Moto 6.5,1999,649.0,42.0,160.0,6.10,NaN
841,Aprilia,Aprilia Pegaso 650,1993,652.0,48.0,160.0,5.22,160.0
...,...,...,...,...,...,...,...,...
38011,Yamaha,Yamaha Zuma 50FX,2016,49.0,NaN,NaN,1.78,NaN
38012,Yamaha,Yamaha Zuma 50FX,2017,49.0,NaN,NaN,1.78,NaN
38013,Yamaha,Yamaha Zuma 50FX,2018,49.0,NaN,NaN,1.78,NaN
38014,Yamaha,Yamaha Zuma 50FX,2019,49.0,NaN,NaN,1.78,NaN
